In [14]:
# Import libraries

import sys
import logging
import pandas as pd
import geopandas as gpd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from model.assumptions_core import read_assumptions
from model.network_core  import build_network
from model.constraints_core import calculate_biogas_max, add_self_sufficiency_constraint, add_battery_flow_constraint, add_biogas_constraint

logging.basicConfig(level=logging.INFO)

In [8]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    "EUR": 10.5080,
    "USD": 9.2521
}
base_year = 2024
discount_rate = 0.04
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3
biogas_method = "average"
weather_start = '2023-01'
weather_end = '2023-12'

## Parameters that will change frequently
geo = '14'
energy_scenario_type = 'fixed'
energy_scenario = 15000000
self_sufficiency = 0.925
target_year = 2030
use_offwind = False
use_h2 = False
h2_initial = 0
biogas_limit = 0.025
growth_only = True


In [11]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_root / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the demand from csv file
if energy_scenario_type == "fixed":
    projected_demand = f"normalized-demand.csv"
    demand = pd.read_csv(paths.demand / projected_demand, index_col = 0)
    demand = demand * energy_scenario
else:
    projected_demand = f"projected-demand,geography={geo},target-year={target_year},growth-only={growth_only}.csv.gz"
    demand = pd.read_csv(paths.demand / projected_demand, index_col = 0, compression='gzip')
    demand = demand * (1 + energy_scenario)

target_load = demand['value'].values.flatten()

# Read the geo, index, and the capacity factors

index = pd.to_datetime(pd.read_csv(paths.weather / f"index,geography={geo},start={weather_start},end={weather_end}.csv")['0'])
geography = gpd.read_file(paths.weather / f"selection,geography={geo},start={weather_start},end={weather_end}.shp").total_bounds
capacity_factor_solar = xr.open_dataarray(paths.renewables / f"capacity-factor-solar,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.renewables / f"capacity-factor-onwind,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.renewables / f"capacity-factor-offwind,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()

In [12]:
# Build the network

biogas = calculate_biogas_max(biogas_limit, target_load, assumptions.loc['combined_cycle_gas_turbine','efficiency'].value, "average")

network = build_network(index, resolution, geography, target_load, assumptions, discount_rate, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [15]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add constraints
add_self_sufficiency_constraint(model, network.loads_t.p_set['load'].values, self_sufficiency)
add_battery_flow_constraint(model, network.links.at["battery-charge", "efficiency"])
add_biogas_constraint(model, network.loads_t.p_set['load'].values, biogas_limit, network.links.at["gas-turbine", "efficiency"])

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 931.14it/s]
INFO:linopy.io: Writing time: 0.49s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 5e+00]
  Cost   [6e+01, 9e+05]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 4e+05]
Presolving model
30668 rows, 27751 cols, 86158 nonzeros  0s
25133 rows, 22216 cols, 79732 nonzeros  0s
24671 rows, 21754 cols, 80656 nonzeros  0s

Solving MIP model with:
   24671 rows
   21754 cols (0 binary, 3 integer, 0 implied int., 21751 continuous)
   80656 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            9913345243.859     Large        0      0      0         0    10.4s
 R       0       0         0   

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32133 primals, 73016 duals
Objective: 9.91e+09
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance, self_sufficiency_constraint, biogas_limit were not assigned to the network.


('ok', 'optimal')

In [16]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator biogas          275.000000                 0.0  6.526469e+05   
          import         2381.617190                 0.0  1.125000e+06   
          onwind         4865.708571                 0.0  7.849225e+06   
          solar          5678.939280                 0.0  5.827358e+06   
Link      AC             3983.893770                 0.0  1.358888e+07   
          li-ion         1476.596458                 0.0  1.971485e+06   
          mixedgas        275.000000                 0.0  3.746193e+05   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     li-ion        13756.064341                 0.0  2.041658e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator biogas    2.184143e-10  6.526469e+05  0.000000e+00         0.270920   
          import    0.000000e+00  1.125000e+06  0.000000e+00         0.053923   
          onwind    1.420708e-11  7.849225e+06  0.000000e+00         0.184152   
          solar     0.000000e+00  5.827358e+06  0.000000e+00         0.117139   
Link      AC        1.367658e+07 -8.769867e+04  1.152890e+07         0.391891   
          li-ion    2.055416e+06 -8.393084e+04  0.000000e+00         0.158904   
          mixedgas  6.526469e+05 -2.780276e+05  6.526469e+05         0.270920   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     li-ion    2.046231e+06 -4.573013e+03  0.000000e+00         0.474405   

                     Curtailment  Capital Expenditure  \
Generator biogas    0.000000e+00         0.000000e+00   
          import    0.000000e+00         0.000000e+00   
          onwind    8.464572e+06         4.429666e+09   
          solar     5.879776e+05         1.988502e+09   
Link      AC        0.000000e+00         2.027658e+08   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         2.530230e+08   
Load      -         0.000000e+00         0.000000e+00   
Store     li-ion    0.000000e+00         1.783000e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator biogas               3.771907e+08      0.0           NaN  
          import               6.750000e+08      0.0           NaN  
          onwind               1.671980e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas             3.108737e+07      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     li-ion              -9.495317e+04      0.0           NaN

In [17]:
generators_t = network.generators_t
generators = network.generators
renewable_generators = ['solar', 'onwind']

curtailed_energy = (generators_t.p_max_pu[renewable_generators].sum() * generators.loc[renewable_generators]['p_nom_opt']).sum() - generators_t.p[renewable_generators].sum().sum()